In [2]:
from typing import List
import torch
import torch.fx
import torch.nn as nn


class ForLoop(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, xs: List[torch.Tensor]):
        while xs[0].numel() > 0:
            xs[0] = torch.zeros((0, 0))
        return xs


for_loop_model = ForLoop()
s_for_loop_model = torch.fx.symbolic_trace(for_loop_model)
# print(s_for_loop_model.graph)


TraceError: symbolically traced variables cannot be used as inputs to control flow

In [1]:
from typing import List
import torch
import torch.nn as nn
from brt.routers.app import RandScatterRouter
from brt.routers import GatherRouter
from brt.frontend import symbolize


class ForLoop(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter = RandScatterRouter(dst_num=2)
        self.moe_decoder = nn.Linear(10, 10)
        self.gather = GatherRouter(dst_num=2)

    def forward(self, x: torch.Tensor):
        target = torch.zeros((1, 1), device=x.device)
        while target.numel() > 0:
            route_results = self.scatter(x)
            target = route_results[0]
            x = self.moe_decoder(route_results[0])
            x = self.gather([x, route_results[1]])
        return x


for_loop_model = ForLoop()

x = torch.randn(1, 10)
y = for_loop_model(x)
print(y)
s_for_loop_model = torch.jit.script(symbolize(for_loop_model))
print(s_for_loop_model.graph)


tensor([[ 0.7801,  1.5037,  0.4237, -1.1433,  0.0262, -0.3877,  0.1027, -0.5722,
          0.5217,  0.2045]], grad_fn=<AliasBackward0>)
graph(%self : __torch__.ForLoop,
      %x.1 : Tensor):
  %8 : NoneType = prim::Constant()
  %2 : int = prim::Constant[value=1]() # /tmp/ipykernel_3032693/2187210996.py:17:30
  %12 : int = prim::Constant[value=9223372036854775807]() # /tmp/ipykernel_3032693/2187210996.py:18:8
  %15 : int = prim::Constant[value=0]() # /tmp/ipykernel_3032693/2187210996.py:18:31
  %5 : Device = prim::device(%x.1)
  %7 : int[] = prim::ListConstruct(%2, %2)
  %target.1 : Tensor = aten::zeros(%7, %8, %8, %5, %8) # /tmp/ipykernel_3032693/2187210996.py:17:17
  %49 : int = aten::numel(%target.1) # /tmp/ipykernel_3032693/2187210996.py:18:14
  %50 : bool = aten::gt(%49, %15) # /tmp/ipykernel_3032693/2187210996.py:18:14
  %x : Tensor = prim::Loop(%12, %50, %x.1) # /tmp/ipykernel_3032693/2187210996.py:18:8
    block0(%18 : int, %x.19 : Tensor):
      %scatter : __torch__.brt.routers